# Terminology and Ontology Project

By 

---- Henri Joël Azemena   azemehenri@gmail.com / azemena1u@etu.univ-lorraine.fr

---- Guillaume Richez    guillaume.richez9@etu.univ-lorraine.fr / guillaume.richez9@etu.univ-lorraine.fr

---- Goal : develop a term identification system for a specific domain.

---- Chosen domain: Natural Language Generation

---- Tools: Linux, Python 3.9, Spacy, textract, os, string, nltk, __future__,plac, random, pathlib, tqdm


### Importing the libraries

In [ ]:
from os import listdir
import textract
import string 
from nltk import word_tokenize
import spacy
from spacy.matcher import Matcher
from nltk import word_tokenize
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm 
from spacy.training.example import Example

## Function that will extract automatically the text  from the set of articles

In [ ]:

def extract_articles_content(path):
    texts = ""
    for file in listdir(path):
        paper = path + file
        if paper != path + '.DS_Store':
            text = textract.process(paper, method='pdfminer',encoding='UTF-8')
            text = text.decode("utf-8") 
            texts = texts + text
        
    return texts

## Creation of our Corpora from the  20  articles in a NLG domain

In [ ]:
corpora = extract_articles_content("path_to_the_scientific_articles_folder")

# Feature engineering 

## Function we use to  clean the corpora 

In [ ]:

def clean_corpora(text_set):
    text_set = text_set.replace('\n' , ' ')
    text_set = text_set.replace('- ' , '')
    text_set = text_set.lower()
    
    #remove punctuation
    
    tokens = word_tokenize(text_set)
    for i,word in enumerate(tokens):
        if 'https' in word or '\\uf' in word:
            del tokens[i]
        if 'hal' in word:
            del tokens[i]
        if 'cid' in word or '¥' in word or '¢' in word:
            del tokens[i]
        if len(word)==1 and word != 'a':
            del tokens[i]
    
    text_set = ' '.join(tokens)
    return text_set

## Clean and save the corpora 

In [ ]:
corpora = clean_corpora(corpora)

with open('corpora.txt','w') as file:
    file.write(corpora)

# Implementation of our SILVER and definition of patterns 

In [ ]:

nlp = spacy.load("en_core_web_sm")
corpora = nlp(corpora)
matcher = Matcher(nlp.vocab)
terms = []
patterns = [[{"POS": "NOUN"}],[{"POS": "NOUN"},{"POS": "NOUN"}],[{"POS": "ADJ"},{"POS": "NOUN"}],[{"POS": "NOUN"},{"POS": "ADJ"}],[{"POS": "NOUN"},{"POS": "ADP"},{"POS": "NOUN"}],
           [{"POS": "NOUN"},{"POS": "ADP"},{"POS": "DET"},{"POS": "NOUN"}],[{"POS": "NOUN"},{"POS": "NOUN"},{"POS": "NOUN"}]]
matcher.add("Compound nouns", patterns)
matches = matcher(corpora)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  
    span = corpora[start:end] 
    terms.append(span.text)


## Clean and save the Silver after manual filterring

In [ ]:
def remove_useless(text_set):
    result = x
    for i,word in enumerate(x):
        if len(word_tokenize(word)) <= 4 or 'CSV' in word or '%' in word:
            del result[i]
        #remove all word with numbers
        if any(char.isdigit() for char in word):
            del result[i]
        if 'CSV Q' in word:
            del result[i]
            
    return result

silver = remove_useless(terms)
silver = set(silver)

In [ ]:
def cleaning_terms(l):
    new_l = [term for term in l if len(term) >= 4 if 'csv' not in term.lower() if '%' not in term.lower()]
    new_l = [term for term in new_l if any(i.isdigit() for i in term) == False]
    new_l = [term for term in new_l if any(token for token in term.split() if len(token) < 2 if len(term) < 10) == False]
    return new_l

len(cleaning_terms(silver))

with open('silver.txt','w') as file:
    for i in silver:
        file.write(i+'\n')


## Load the filter SILVER

In [47]:
silver = []
with open('silver.txt') as file:
    lines = file.readlines()
    
    for line in lines :
        silver.append(line)
silver = [x[:-1] for x in silver]

## Load the copora

In [48]:
with open('corpora.txt') as file:
    corpora = file.readlines()

corpus = corpus[0]

## Creation of IBO finction for automatic ananotation

In [50]:
def annot(text, silver):
    tokens = word_tokenize(text)
    #print(tokens)
    offset = 0
    entities = []
    i = 0
    while i < (len(tokens)):
        offset = text.find(tokens[i], offset)
        if i < len(tokens) - 3 and " ".join(tokens[i:i+3]) in silver:
            entities.append((offset,offset+len(tokens[i]),'B'))
            entities.append((offset+len(tokens[i])+1,offset+len(tokens[i])+len(tokens[i+1])+1,'I'))
            entities.append((offset+len(tokens[i])+len(tokens[i+1])+2,offset+len(tokens[i])+len(tokens[i+1])+len(tokens[i+2])+2,'I'))
            i = i+3
            #offset=offset+len(tokens[i])+len(tokens[i+1])+len(tokens[i+2])
        elif i < len(tokens) - 2 and " ".join(tokens[i:i+2]) in silver:
            entities.append((offset,offset+len(tokens[i]),'B'))
            entities.append((offset+len(tokens[i])+1,offset+len(tokens[i])+len(tokens[i+1])+2,'I'))
            i = i+2 
            #offset=offset+len(tokens[i])+len(tokens[i+1])
        elif i < len(tokens) - 1 and tokens[i] in silver:
            entities.append((offset,offset+len(tokens[i]),'B'))
            i = i+1
            #offset=offset+len(tokens[i])
        else:
            entities.append((offset,offset+len(tokens[i]),'O'))
            i = i+1
        offset = entities[-1][1]
    return {'entities': entities}


## Annotation of the corpora using the filter SILVER

In [51]:
tokens = word_tokenize(corpora)
TRAIN_DATA = []
for i in range(0,len(tokens),20):
    text = " ".join(tokens[i:i+20])
    TRAIN_DATA.append((text, annot(text, silver)))


## Saving the training set

In [52]:
with open('traindata.txt','w') as file:
    for a, annotations in TRAIN_DATA:
        record = "'"+a + "'," + str(annotations) +'\n'
        file.write(record)

# NER setting using Spacy

## Creation of our Model 

In [54]:
model_ner = None
output_dir=Path("path_to_the_folder_where_we want_to_save_the_model")
n_iter=150

In [55]:
if model_ner is not None:
    nlp = spacy.load(model_ner)  
    print("Loaded model '%s'" % model_ner)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model_ner")

Created blank 'en' model


In [56]:
if "ner" not in nlp.pipe_names:
        #ner = nlp.create_pipe("ner")
        ner = nlp.add_pipe("ner", last=True)
    # otherwise, get it so we can add labels
else:
        ner = nlp.get_pipe("ner")

## Training of our model

In [ ]:

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

  0%|          | 1/2209 [00:00<03:57,  9.29it/s]/Users/lasirene/opt/anaconda3/lib/python3.7/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "recognition asr errors and out-of-vocabulary oov p..." with entities "[(0, 11, 'B'), (12, 15, 'B'), (16, 23, 'I'), (23, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
  0%|          | 5/2209 [00:00<02:16, 16.14it/s]/Users/lasirene/opt/anaconda3/lib/python3.7/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "pluridisciplinaire est destinée au dépôt et la dif..." with entities "[(0, 18, 'B'), (19, 22, 'O'), (23, 31, 'O'), (32, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned ent

{'ner': 10673.924832065906}


  0%|          | 2/2209 [00:00<02:01, 18.16it/s]

{'ner': 6273.9614340944545}


  0%|          | 2/2209 [00:00<02:17, 16.07it/s]

...

  0%|          | 3/2209 [00:00<01:37, 22.66it/s]

{'ner': 1923.5555158444035}


  0%|          | 3/2209 [00:00<01:30, 24.27it/s]

{'ner': 1883.3334529966503}


 22%|██▏       | 492/2209 [00:21<01:12, 23.73it/s]

## Saving of the Model 

In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)    

# Loading and using the created model on new data 

In [ ]:
corpora_validation = corpora = extract_articles_content("path_to_the_scientific_articles_test_folder")

In [ ]:
nlp_validation = spacy.load("the value of output_dir = Pathpath_to_the_folder_where_we_save_the_model")
doc_validation = nlp_validation(corpora_validation)
spacy.displacy.render(doc_validation, style="ent", jupyter=True)

In [ ]:
annot(corpora_validation, silver)

In [ ]:
nlp_validation = spacy.load("en_core_web_sm")
doc_validation = nlp_validation(corpora_validation[0])

for token in doc_validation:
   print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop + len(token.text) - 1)

# Conclusion

In this project, we learned to recognize from a set of documents, the entities belonging to a specific domain by using the basic principles of terminology. To optimize the result, we can increase the number of epochs.
